Getting award names

In [423]:
from collections import Counter
import re
import json 
import os

def load_json(filename):
    path = os.path.join(os.getcwd(), filename)
    with open(path, 'r') as f:
        data = json.load(f)
    return data

# loading the data
data = load_json("/Users/gustavolucasdecarvalho/Desktop/CS_337/gg2013.json")

# data without retweets
data_no_rt = []
for tw in data:
    text = tw["text"]

    if not re.search("^RT\s", text):
        data_no_rt.append(tw)
        
# MY CODE NEEDS THE NAME OF THE AWARDS SO MAKE SURE TO INCLUDE THIS
award_name = "golden globes"

# getting the tweets that start with "Best" and continue on with another word with a capital letter
has_best = []
for tw in data_no_rt:
    text = tw["text"]

    if re.search("^Best\s[A-Z].+", text):
        has_best.append(text)

awards_d = Counter(has_best)
print(len(awards_d.keys()))

## further filtering idea by grabbing relevant sentence structure
a_lst = []
for k in awards_d.keys():
    goes_to = re.search("[gG]oes [Tt]o",k)
    if goes_to:
        a_lst.append(k[:goes_to.start()].strip())

    colon = re.search(":\s",k)
    if colon:
        a_lst.append(k[:colon.start()].strip())

    dash = re.search("^Best\s[A-Z].+\s-",k)
    if dash:
        a_lst.append(k[:dash.end()-2].strip())

awards_d = Counter(a_lst)
print(len(awards_d.keys()))

# deleting tweets which fit one of a given set of heuristics that I found to be a good way to filter out tweets
del_lst = []
for k, v in awards_d.items():
    sen = k.split(" ")
    bools = (len(sen) > 9) or (v < 2) or (award_name in k.lower()) or (award_name[:-1] in k.lower()) or re.search("[^\w\s]", k)#or (':' in k) or (',' in k) or ('-' in k)
    if bools: del_lst.append(k)
for k in del_lst:
    del awards_d[k]

print(len(awards_d.keys()))
awards_d.most_common()

# replace words with synonyms to create more explicit duplicates which are then removed
new_awards_d = Counter()
for k, v in awards_d.items():
    k = k.replace("TV Series", "Television")
    k = k.replace(" TV ", " Television ")
    k = k.replace(" TV", " Television")

    k = k.replace(" Movie ", " Motion Picture ")
    k = k.replace(" Movie", " Motion Picture")

    k = k.replace("Mini Series", "Miniseries")
    k = k.replace("Mini series", "Miniseries")
    k = k.replace("mini series", "Miniseries")

    k = k.replace("Foreign Film", "Foreign Language Film")
    k = k.replace( "Feature Film", "Film")

    k = k.replace("Performance by an", "")
    k = k.replace("  ", " ")
    
    new_awards_d[k] += v

awards_d = new_awards_d
print(len(awards_d.keys()))

# removing duplicates which only differ by spaces and/or symbols
awards_set = Counter()
awards_d_to_s = {}
for k, v in awards_d.items():
    txt_lst = re.split("/W", k)
    txt_lst.sort()
    txt = " ".join(txt_lst).strip()
    
    awards_d_to_s[k] = txt
    awards_set[txt] += v

seen = set()
new_awards_d = Counter()
for k, v in awards_d.items():
    txt = awards_d_to_s[k]
    if txt not in seen:
        seen.add(txt)
        new_awards_d[k] = awards_set[txt]

awards_d = new_awards_d
print(len(awards_d.keys()))

# deleting more duplicates
del_set = set()
for k in awards_d.keys():
    k_lst = k.split(" ")
    for k2 in awards_d.keys():
        k2_lst = k2.split(" ")

        if len(k_lst) >= len(k2_lst):
            continue
        elif k_lst == k2_lst[:len(k_lst)]:
            del_set.add(k)
for d in del_set:
    del awards_d[d]

print(len(awards_d.keys()))

# printing list of awards
awards_lst = list(awards_d.keys())
for a in awards_lst:
    print(a)

715
395
41
35
35
24
Best Drama
Best Supporting Actor in a Motion Picture
Best Supporting Actor in a Drama
Best Miniseries or Television Motion Picture
Best Television Motion Picture or Miniseries
Best Miniseries or Motion Picture Made for Television
Best Actress in a Miniseries or Television Motion Picture
Best Actor in a Television Series
Best Television Drama
Best Original Score
Best Original Song
Best Supporting Actress in a Motion Picture
Best Screenplay Motion Picture
Best Actor in a Comedy
Best Foreign Language Film
Best Actress Television Drama
Best Animated Film
Best Actress in a Television Comedy
Best Director
Best Motion Picture Comedy or Musical
Best Picture
Best Actress in a Motion Picture Drama
Best Actor in a Motion Picture Drama
Best Motion Picture Drama


Getting best dressed

In [368]:
has_best = []
for tw in data_no_rt:
    text = tw["text"]

    if re.search("b+e+a+u+t+i+f+u+l+|g+o+a+r+g+e+o+u+s+|f+a+b+u+l+o+u+s+|s+e+x+y+", text):
        text = text.replace("@", "")
        has_best.append(text)

dress_tw = Counter(has_best)
dress_tw.most_common()

[('Kate Hudson looks beautiful #GoldenGlobes', 2),
 ('Life of pi was beautiful.. Really enjoyed that movie.#goldenglobes', 2),
 ('Kerry Washington looks fabulous!! #ERedCarpet #GoldenGlobes #wendyredcarpet',
  1),
 ('Agree! RT Danishaz: Sofia Vergara trop marrante et sexy #goldenglobes', 1),
 ('George Clooney will be a hottie at 90 years old I swear! That man looks goooood! #GoldenGlobes #sexy',
  1),
 ('Why does jLO make that face?? U know the "I think I\'m so sexy" kissy face? #annoying but that dress.. Hmmm.. #goldenglobes',
  1),
 ('Anne Hathaway looks beautiful as always. #ERedCarpet goldenglobes', 1),
 ("Yes. He is. “ReignOfApril: Clooney can't get his jacket closed.  I wonder what's up with that.  He's so beautiful.  #GoldenGlobes”",
  1),
 ('Jessica Chastain se ve sexy, pero ese vestido no me gusta casi #GoldenGlobes',
  1),
 ('Denzel Washington with his beautiful daughter too sweet #ERedCarpet #GoldenGlobes http://t.co/CG30b1rp',
  1),
 ('Leonardo DiCaprio spotted on the red c